In [2]:
import random

# *single*

In [3]:
data = [1, 2, 3, 5, 6, 7]
lables = [155, 197, 244, 356, 407, 448]

target = 4

In [4]:
def squ(m, b, x, y, rate):
    y_hat = m * x + b
    m += rate * (y - y_hat) * x
    b += rate * (y - y_hat)
    return m, b

In [96]:
def abs(m, b, x, y, rate):
    y_hat = m * x + b
    if y_hat < y:
        m += rate * x
        b += rate
    else:
        m -= rate * x
        b -= rate
    return m, b

In [ ]:
def linear(feat, lab, rate = 0.01, epos = 50, trick = 'squ'):
    m = random.random()
    b = random.random()
    for epo in range(epos):
        i = random.randint(0, len(feat) - 1)
        x = feat[i]
        y = lab[i]
        if trick == 'squ':
            m, b = squ(m, b, x, y, rate)
        elif trick == 'abs':
            m, b = abs(m, b, x, y, rate)
    return m, b

In [47]:
m, b = linear(data, lables, rate = 0.04, epos = 1000, trick = 'squ')

print(f'm = {m}, b = {b}')
print(m * target + b)

m = 51.84311599768799, b = 100.30660523869903
307.679069229451


In [107]:
m2, b2 = linear(data, lables, rate = 0.04, epos = 100000, trick = 'abs')

print(f'm = {m2}, b = {b2}')
print(m2 * target + b2)

m = 50.92104683456431, b = 97.16085525001462
300.84504258827184


# *multi*

In [58]:
#rooms, pool, area
data_multi = [[1, 1, 300], [2, 0, 200], [3, 1, 450], [5, 1, 500], [6, 0, 500], [7, 1, 800]]
lables = [155, 197, 244, 356, 407, 448]

target_multi = [4, 1, 600]

In [70]:
def squ(m, b, x, y, rate):
    for i in range(len(x)):
        y_hat = m[i] * x[i]
    y_hat += b

    for i in range(len(m)):
        m[i] += rate * (y - y_hat) * x[i]
        
    b += rate * (y - y_hat)
    return m, b

In [71]:
def linear_multi(feat, lab, rate = 0.01, epos = 50):
    m = [random.random() for _ in range(len(feat[0]))] 
    b = random.random()
    for epo in range(epos):
        i = random.randint(0, len(feat) - 1)
        x = feat[i]
        y = lab[i]
        m, b = squ(m, b, x, y, rate)
    return m, b

In [84]:
m_multi, b_multi = linear_multi(data_multi, lables, rate = 0.000001, epos = 10000)

print(f'm = {m_multi}, b = {b_multi}') # m_multi is a list of wegiths, b_multi is a bias

prediction = sum(m_multi[i] * target_multi[i] for i in range(len(target_multi))) + b_multi
print(f'prediction = {prediction}')

m = [1.139676353274608, -0.12975714655258647, 0.5585149750999978], b = 0.40665458123401194
prediction = 339.94458790777855


# *gradiant*

# *logistic*

In [108]:
import numpy as np

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def prediction(m, b, x):
    y_hat = sum(m[i] * x[i] for i in range(len(x))) + b
    return sigmoid(y_hat)

def log_loss(y, y_hat):
    return - (y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))

def logistic(feat, lab, rate=0.01, epos=50):
    m = np.random.rand(len(feat[0]))
    b = random.random()
    
    for epo in range(epos):
        i = random.randint(0, len(feat) - 1)
        x = feat[i]
        y = lab[i]
        y_hat = prediction(m, b, x)
        
        m += rate * (y - y_hat) * y_hat * (1 - y_hat) * np.array(x)
        b += rate * (y - y_hat) * y_hat * (1 - y_hat)
    return m, b

In [126]:
feat = [[1, 0], [3, 2], [1, 1], [4, 2], [1, 3], [2, 2], [2, 3], [3, 2]]
lab = [1, 1, 0, 1, 0, 0, 0, 1]

m, b = logistic(feat, lab, rate=0.01, epos=100000)
print(f'm = {m}, b = {b}')

losses = [log_loss(y, prediction(m, b, x)) for x, y in zip(feat, lab)]
print("Final average log-loss:", np.mean(losses))

m = [ 4.390546   -4.41755702], b = -1.9558613684451724
Final average log-loss: 0.06480444115353645


In [132]:
# in [a, b] if (a > b) y = 1 else 0
new_feat = [[1, 2], [7, 5], [5, 5]]

for x in new_feat:
    prob = prediction(m, b, x)
    pred_class = 1 if prob >= 0.5 else 0
    print(f"Input: {x}, Prob(class=1): {prob:.4f}, Predicted class: {pred_class}")

Input: [1, 2], Prob(class=1): 0.0017, Predicted class: 0
Input: [7, 5], Prob(class=1): 0.9988, Predicted class: 1
Input: [5, 5], Prob(class=1): 0.1100, Predicted class: 0
